In [1]:
import pandas as pd
import re

# Cargar archivos IOS con codificación ISO-8859-1 y delimitador ';'
ios_files = ['IOS01.csv', 'IOS03.csv', 'IOS05.csv', 'IOS07.csv', 'IOS09.csv']

# Cargar y concatenar los archivos IOS
ios_data_list = []
for f in ios_files:
    try:
        data = pd.read_csv(f, encoding='ISO-8859-1', delimiter=';', on_bad_lines='skip')
        ios_data_list.append(data)
        print(f"Archivo {f} cargado correctamente con {len(data)} filas.")
    except Exception as e:
        print(f"Error al cargar el archivo {f}: {e}")

# Concatenar todos los datos de los archivos IOS
ios_data = pd.concat(ios_data_list, ignore_index=True)

# Verificar las primeras filas para asegurarnos que los datos se han cargado correctamente
print("Primeras filas de los archivos IOS cargados:")
print(ios_data.head())

# Limpiar los nombres de las columnas de los archivos IOS
ios_data.columns = ios_data.columns.str.strip()

# Filtrar los datos de los archivos IOS según las condiciones dadas
modulos_validos = ['ICM1', 'ICM2', 'ICM3', 'ICM4', 'AIN']
# Filtrar descripciones que contengan "Inicio de falla" en cualquier parte de la cadena
ios_data = ios_data[ios_data['Nombre del módulo'].isin(modulos_validos) &
                    ios_data['Descripción'].str.contains("Inicio de falla", case=False, na=False)]

# Eliminar filas con '#N/D' en la columna 'Descripción'
ios_data = ios_data[ios_data['Descripción'] != '#N/D']

# Eliminar columnas innecesarias
columnas_a_eliminar = ['Usuario', 'MsgID', 'Valor nuevo', 'Nombre del atributo']
ios_data = ios_data.drop(columns=[col for col in columnas_a_eliminar if col in ios_data.columns])

# Agregar las columnas constantes "Planta" y "Area"
ios_data['Planta'] = 'Tuxtepec'
ios_data['Area'] = 'Brewing'

# Verificar el número de filas después de los filtros
print(f"Filas después de aplicar filtros de módulos y descripciones: {len(ios_data)}")

# Cargar archivos MATRIZ y EXCEPCIONES con codificación UTF-8 y delimitador ','
try:
    matriz_data = pd.read_csv('MATRIZ.csv', encoding='UTF-8', delimiter=',')
    print(f"Archivo MATRIZ cargado correctamente con {len(matriz_data)} filas.")
except Exception as e:
    print(f"Error al cargar el archivo MATRIZ: {e}")

# Leer excepciones desde un archivo .txt
try:
    with open('EXCEPCIONES.txt', 'r') as file:
        # Leer cada línea y eliminar espacios en blanco o saltos de línea
        equipos_excepciones = [line.strip() for line in file.readlines()]
    print(f"Archivo EXCEPCIONES.txt cargado correctamente con {len(equipos_excepciones)} excepciones.")
except Exception as e:
    print(f"Error al cargar el archivo EXCEPCIONES.txt: {e}")

# Crear un patrón que combine todas las excepciones
excepciones_patron = '|'.join(re.escape(excepcion) for excepcion in equipos_excepciones)

# Filtrar las filas donde 'Nombre de objeto' no contenga ninguna excepción
ios_data = ios_data[~ios_data['Nombre de objeto'].str.contains(excepciones_patron, na=False)]

# Verificar el número de filas después de aplicar el filtro de excepciones
print(f"Filas después de eliminar los equipos de EXCEPCIONES: {len(ios_data)}")

# Integrar los datos de MATRIZ (relación de etiquetas TAG con AREA)
def agregar_area_desde_matriz(descripcion):
    # Buscar si el TAG (fragmento) está en la columna TAG de MATRIZ
    for tag in matriz_data['TAG']:
        if tag in descripcion:
            return matriz_data[matriz_data['TAG'] == tag]['AREA'].values[0]
    return None

# Aplicar la función de integración para agregar la columna AREA
ios_data['AREA_MATRIZ'] = ios_data['Nombre de objeto'].apply(agregar_area_desde_matriz)

# Ordenar los datos por "Fecha y Hora" de manera descendente sin convertir la columna
ios_data = ios_data.sort_values('Fecha y Hora', ascending=False)

# Renombrar las columnas según el mapeo proporcionado
ios_data = ios_data.rename(columns={
    'Número PCU': 'PLC',
    'Nombre de objeto': 'Tag',
    'Nombre de Unidad': 'Group',
    'Nombre de categoría de receta maestra': 'Receta',
    'Nombre del módulo': 'Objeto',
    'AREA_MATRIZ': 'Subarea',
    'Descripción': 'Descripcion'
})

# Verificar los nombres de las columnas después de renombrarlas
print("Nombres de columnas después de renombrar:")
print(ios_data.columns)

# Reordenar las columnas según el orden final solicitado
ios_data = ios_data[['Planta', 'Area', 'Subarea', 'PLC', 'Tag', 'Group', 'Receta', 'Objeto', 'Fecha y Hora', 'Descripcion']]

# Eliminar filas donde la columna 'Subarea' tenga valores en blanco o nulos
ios_data = ios_data.dropna(subset=['Subarea'])

# Guardar el DataFrame resultante en un archivo CSV con codificación UTF-8
ios_data.to_csv('resultado_filtrado_final.csv', encoding='UTF-8', index=False)

# Verificar las primeras filas para asegurar que todo está correcto
print("Primeras filas del archivo final generado:")
print(ios_data.head())

# Guardar el DataFrame resultante en un archivo CSV con codificación UTF-8
ios_data.to_csv('TablaFallas.csv', encoding='UTF-8', index=False)

# Verificar las primeras filas para asegurar que todo está correcto
print("Primeras filas del archivo final generado:")
print(ios_data.head())

import pandas as pd
import time
from IPython.display import FileLink

# Paso 1: Leer el archivo CSV original
input_csv = 'TablaFallas.csv'  # Ruta del archivo original
output_csv_mod = 'TablaFallas_modificado.csv'  # Ruta para el archivo modificado
output_csv_no_empty = 'TablaFallas_sin_vacios.csv'  # Ruta para el archivo sin vacíos
output_txt_line_protocol = 'TablaFallas_line_protocol.txt'  # Ruta para el archivo Line Protocol
output_txt_adjusted = 'Fallas.txt'  # Ruta para el archivo ajustado con timestamps

# Cargar los datos del CSV original
df = pd.read_csv(input_csv)

# Paso 2: Eliminar columnas no deseadas y renombrar 'Fecha y Hora' a 'Fecha'
df.drop(columns=['Descripcion'], inplace=True)
df.rename(columns={'Fecha y Hora': 'Fecha'}, inplace=True)

# Guardar el DataFrame modificado
df.to_csv(output_csv_mod, index=False, encoding='utf-8')
print("Paso 2: Columnas eliminadas y columna 'Fecha' renombrada.")

# Paso 3: Reemplazar valores vacíos con 'Desconocido'
df = pd.read_csv(output_csv_mod)
df.fillna("Desconocido", inplace=True)

# Guardar el DataFrame sin vacíos
df.to_csv(output_csv_no_empty, index=False, encoding='utf-8')
print("Paso 3: Campos vacíos reemplazados por 'Desconocido'.")

# Paso 4: Convertir datos a Line Protocol
df = pd.read_csv(output_csv_no_empty)
line_protocol_lines = []

# Función para escapar espacios
def escape_spaces(name):
    return str(name).replace(" ", "\\ ")

# Recorrer cada fila para crear líneas de Line Protocol
for index, row in df.iterrows():
    planta = escape_spaces(row['Planta'])
    area = escape_spaces(row['Area'])
    subarea = escape_spaces(row['Subarea'])
    plc = escape_spaces(row['PLC'])
    grupo = escape_spaces(row['Group'])
    receta = row['Receta']
    objeto = row['Objeto']
    tag = row['Tag']

    timestamp = int(time.mktime(time.strptime(row['Fecha'], "%d.%m.%Y %H:%M:%S"))) * 1_000_000_000

    line = (
        f"fallas,Planta={planta},Area={area},Subarea={subarea},PLC={plc},Group={grupo} "
        f"Tag=\"{tag}\",Objeto=\"{objeto}\",Receta=\"{receta}\" {timestamp}"
    )
    line_protocol_lines.append(line)

# Guardar Line Protocol en un archivo .txt
with open(output_txt_line_protocol, 'w', encoding='utf-8') as f:
    for line in line_protocol_lines:
        f.write(line + "\n")

print("Paso 4: Datos convertidos a Line Protocol y guardados en archivo.")

# Paso 5: Ajustar timestamps repetidos
timestamp_count = {}
adjusted_lines = []

with open(output_txt_line_protocol, 'r', encoding='utf-8') as f:
    lines = f.readlines()

for line in lines:
    parts = line.strip().rsplit(' ', 1)
    if len(parts) != 2:
        continue

    line_data, timestamp_str = parts
    timestamp = int(timestamp_str)

    if timestamp in timestamp_count:
        timestamp_count[timestamp] += 1
        adjusted_timestamp = timestamp + timestamp_count[timestamp]
    else:
        timestamp_count[timestamp] = 0
        adjusted_timestamp = timestamp

    adjusted_line = f"{line_data} {adjusted_timestamp}"
    adjusted_lines.append(adjusted_line)

# Guardar las líneas ajustadas
with open(output_txt_adjusted, 'w', encoding='utf-8') as f:
    for adjusted_line in adjusted_lines:
        f.write(adjusted_line + "\n")

print("Paso 5: Timestamps ajustados y guardados en archivo.")

# Paso 6: Generar enlace para descargar el archivo ajustado
print(f"Paso 6: Archivo ajustado disponible para descarga:")
display(FileLink(output_txt_adjusted))


Archivo IOS01.csv cargado correctamente con 24533 filas.
Archivo IOS03.csv cargado correctamente con 42680 filas.
Archivo IOS05.csv cargado correctamente con 32910 filas.
Archivo IOS07.csv cargado correctamente con 20481 filas.
Archivo IOS09.csv cargado correctamente con 17785 filas.
Primeras filas de los archivos IOS cargados:
   Número PCU Nombre de objeto  Nombre de Unidad  \
0           0              NaN  IOS           21   
1           0              NaN  IOS           22   
2           0              NaN  IOS           21   
3           0              NaN  IOS           22   
4           0              NaN  IOS           22   

  Nombre de categoría de receta maestra Nombre del módulo  \
0                                 Login               IOS   
1                                 Login               IOS   
2                                Logout               IOS   
3                                Logout               IOS   
4                                 Login             

/content/Fallas.txt